# Import General Packages

In [1]:
# General Packages
import pandas as pd
import numpy as np
import re
import os
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
from datasets import Dataset, load_dataset
import pandas as pd

/media/dzielinski06/HDD1/AI894 - Capstone/Complete Collision Recorder/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# import os
# import sys
# import logging
# import warnings
# from pathlib import Path

# import torch
# # import intel_extension_for_pytorch as ipex
# from datasets import load_dataset
# from transformers import (
#     AutoModelForCausalLM,
#     AutoTokenizer,
#     TrainingArguments,
#     Trainer,
#     DataCollatorForSeq2Seq,
# )
# from peft import LoraConfig, get_peft_model

# # # Constants
# BASE_MODEL = "mistralai/Mistral-7B-v0.1"
# DATA_PATH = "b-mc2/sql-create-context"
# MODEL_PATH = "./final_model"
# ADAPTER_PATH = "./lora_adapters"
# DEVICE = torch.device("xpu" if torch.xpu.is_available() else "cpu")
# MODEL_CACHE_PATH = "/home/common/data/Big_Data/GenAI/llm_models"

# # # Weights & Biases Configuration
# ENABLE_WANDB = False

# if ENABLE_WANDB:
#     print("installing wandb...")
#     !{sys.executable} -m pip install -U --force "wandb==0.15.12" > /dev/null 2>&1
#     print("installation complete...")

#     import wandb
#     os.environ["WANDB_NOTEBOOK_NAME"] = os.path.abspath('')
#     os.environ["WANDB_PROJECT"] = f"finetune-model-name_{BASE_MODEL.replace('/', '_')}"
#     os.environ["WANDB_LOG_MODEL"] = "checkpoint"
#     wandb.login()

# # # Configuration
# warnings.filterwarnings("ignore", category=UserWarning)
# os.environ["TOKENIZERS_PARALLELISM"] = "false"
# os.environ["NUMEXPR_MAX_THREADS"] = "10"
# os.environ["ENABLE_SDP_FUSION"] = "true"
# os.environ["SYCL_PI_LEVEL_ZERO_USE_IMMEDIATE_COMMANDLISTS"] = "1"

# logging.getLogger("transformers").setLevel(logging.ERROR)

# # Map Directories and Import NLTK Data

In [3]:
# Map src directory
import sys
import os
root_dir = os.path.abspath(os.path.join(os.getcwd(), "../."))
print("Root Directory: ", root_dir)
src_dir = os.path.join(root_dir,"src")
print("Src Directory: ", src_dir)
sys.path.append(src_dir)
data_dir = os.path.join(root_dir,"data")
print("Data Directory: ", data_dir)

Root Directory:  /media/dzielinski06/HDD1/AI894 - Capstone/Complete Collision Recorder
Src Directory:  /media/dzielinski06/HDD1/AI894 - Capstone/Complete Collision Recorder/src
Data Directory:  /media/dzielinski06/HDD1/AI894 - Capstone/Complete Collision Recorder/data


In [4]:
from torch.utils.data import Dataset, DataLoader

# Import and Transform Data

In [5]:
# from transformers import pipeline

# # Check if a GPU is available
# device = 0 if torch.cuda.is_available() else -1

# # Initialize the summarizer with the device argument
# summarizer = pipeline('text2text-generation', model='describeai/gemini', device=device)

# code = "print('hello world!')"
# response = summarizer(code, max_length=100, num_beams=3)

# print("Summarized code: " + response[0]['generated_text'])


In [5]:
def concatenate_texts(row):
    cad_text = row['CAD_TEXT'] if pd.notna(row['CAD_TEXT']) else ""
    oh_text = row['OH1_TEXT'] if pd.notna(row['OH1_TEXT']) else ""
    
    if oh_text:  # If 'OH_TEXT' is not an empty string
        return cad_text + "POLICE NARRATIVE \n\n " + oh_text
    else:
        return cad_text  # If 'OH_TEXT' is empty, return only 'CAD_TEXT'

# Apply the function to the dataframe to create a new concatenated column
training_df = pd.read_csv(os.path.join(data_dir, "processed", "training_df.csv"))
training_df['concatenated_text'] = training_df.apply(concatenate_texts, axis=1)
training_df = training_df[training_df["BIKE_CLE_TEXT"].notnull() & training_df["concatenated_text"].notnull()]
training_df = training_df.dropna(subset=['concatenated_text', 'BIKE_CLE_TEXT'])
training_df = training_df[training_df['concatenated_text'].str.strip() != '']
training_df = training_df[training_df['BIKE_CLE_TEXT'].str.strip() != '']

In [6]:
# training_df

In [7]:
class CustomDataset(Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels

    def __len__(self):
        return len(self.inputs['input_ids'])

    def __getitem__(self, idx):
        return {
            'input_ids': self.inputs['input_ids'][idx],
            'attention_mask': self.inputs['attention_mask'][idx],
            'labels': self.labels['input_ids'][idx]  # Use input_ids for labels
        }

In [9]:
# import torch
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments

# tokenizer = AutoTokenizer.from_pretrained("describeai/gemini")
# model = AutoModelForSeq2SeqLM.from_pretrained("describeai/gemini")
# # model.parallelize() 
# model.gradient_checkpointing_enable()
# model = torch.nn.DataParallel(model)

# # Ensure that all inputs are on the same device as the model
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)

# # Move the inputs and labels to the same device
# def move_batch_to_device(batch, device):
#     return {key: value.to(device) for key, value in batch.items()}

# inputs = tokenizer(list(training_df['concatenated_text']), padding=True, truncation=True, return_tensors="pt")
# labels = tokenizer(list(training_df['BIKE_CLE_TEXT']), padding=True, truncation=True, return_tensors="pt")
# dataset = CustomDataset(inputs, labels)

# batch_size = 4
# train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# training_args = TrainingArguments(
#     output_dir="./results",
#     per_device_train_batch_size=batch_size,
#     num_train_epochs=3,
#     logging_dir="./logs",
#     logging_steps=10,
#     save_steps=100,
#     fp16=True  # Enable mixed precision if needed
# )

# for batch in train_loader:
#     batch = move_batch_to_device(batch, device)
#     break  # Just to check one batch


# # Now train the model with accelerate handling multi-GPU setup
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=dataset,
# )

# trainer.train()

# CPU Deployment

In [10]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM

# config = PeftConfig.from_pretrained("spikecodes/ai-911-operator")
base_model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1")
model = PeftModel.from_pretrained(base_model, "spikecodes/ai-911-operator")

Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.97s/it]


: 

In [11]:
import torch
from transformers import AutoTokenizer, Trainer, TrainingArguments



In [12]:

# longest_observation = training_df['concatenated_text'].loc[training_df['concatenated_text'].str.len().idxmax()]

average_length = training_df['concatenated_text'].str.len().mean()
average_length

3455.361216730038

In [13]:
# def setup_model_and_tokenizer(base_model_id: str):
#     local_model_id = base_model_id.replace("/", "--")
#     local_model_path = os.path.join(MODEL_CACHE_PATH, local_model_id)
    
#     try:
#         print(f"Attempting to load model and tokenizer from: {local_model_path}")
#         model = AutoModelForCausalLM.from_pretrained(
#             local_model_path,
#             torch_dtype=torch.bfloat16,
#             low_cpu_mem_usage=True,
#         )
#         tokenizer = AutoTokenizer.from_pretrained(local_model_path)
#     except (OSError, PermissionError) as e:
#         print(f"Failed to load from {local_model_path}. Attempting to download...")
#         model = AutoModelForCausalLM.from_pretrained(
#             base_model_id,
#             torch_dtype=torch.bfloat16,
#             low_cpu_mem_usage=True,
#         )
#         tokenizer = AutoTokenizer.from_pretrained(base_model_id)

#     tokenizer.pad_token_id = 0
#     tokenizer.padding_side = "left"
#     return model, tokenizer

In [14]:

# import pandas as pd
# from datasets import Dataset
# from transformers import DataCollatorForLanguageModeling

# def generate_prompt_911(messages):
#     prompt = "You are reviewing descriptions of collisions between vehicles and pedestrians or bicycles. Your job is to summarize narratives from dispatch reports and police narratives.\n\n"
#     for message in messages:
#         role = "Reviewer" if message['role'] == 'assistant' else "CAD"
#         prompt += f"{role}: {message['content']}\n"
#     prompt += "Reviewer:"
#     return prompt

# class FineTuner:
#     def __init__(self, base_model_id: str, model_path: str, device: torch.device):
#         self.base_model_id = base_model_id
#         self.model_path = model_path
#         self.device = device
#         self.model, self.tokenizer = setup_model_and_tokenizer(base_model_id)

#     # def tokenize_data(self, data_point, add_eos_token=True, cutoff_len=512):
#     #     prompt = generate_prompt_911(data_point["messages"])
#     #     tokenized = self.tokenizer(
#     #         prompt,
#     #         truncation=True,
#     #         max_length=cutoff_len,
#     #         padding=False,
#     #         return_tensors=None,
#     #     )
        
#     #     if (
#     #         tokenized["input_ids"][-1] != self.tokenizer.eos_token_id
#     #         and add_eos_token
#     #         and len(tokenized["input_ids"]) < cutoff_len
#     #     ):
#     #         tokenized["input_ids"].append(self.tokenizer.eos_token_id)
#     #         tokenized["attention_mask"].append(1)
            
#     #     tokenized["labels"] = tokenized["input_ids"].copy()
#     #     return tokenized
    
#     def tokenize_data(self, data_batch, add_eos_token=True, cutoff_len=512):
#         # Generate prompts for the batch (list of texts)

#         if isinstance(data_batch, dict):  # Single data point
#             data_batch = [data_batch]


#         prompts = [generate_prompt_911(data_point["messages"]) for data_point in data_batch]
        
#         # Batch tokenization (tokenizing all prompts in parallel)
#         tokenized_batch = self.tokenizer(
#             prompts,
#             truncation=True,
#             max_length=cutoff_len,
#             padding="max_length",  # Use padding to the max length of the batch
#             return_tensors="pt",  # Return PyTorch tensors for batch processing
#         )
        
#         # Check for EOS token and handle it in each sequence
#         # if add_eos_token:
#         #     for idx, input_ids in enumerate(tokenized_batch["input_ids"]):
#         #         if (
#         #             input_ids[-1] != self.tokenizer.eos_token_id
#         #             and len(input_ids) < cutoff_len
#         #         ):
#         #             tokenized_batch["input_ids"][idx][-1] = self.tokenizer.eos_token_id
#         #             tokenized_batch["attention_mask"][idx][-1] = 1
        
#         # Labels should be a copy of input_ids for causal language models
#         tokenized_batch["labels"] = tokenized_batch["input_ids"].clone()

#         return tokenized_batch



#     # def prepare_data(self, data, val_set_size=100):
#     #     # train_val_split = data["train"].train_test_split(
#     #     #     test_size=val_set_size, shuffle=True, seed=123
#     #     # )
#     #     # train_data = train_val_split["train"].shuffle().map(self.tokenize_data)
#     #     # val_data = train_val_split["test"].shuffle().map(self.tokenize_data)

#     #     train_val_split = data.train_test_split(
#     #         test_size=val_set_size, shuffle=True, seed=123)

#     #     train_data = train_val_split["train"].shuffle(seed=123).map(self.tokenize_data)
#     #     val_data = train_val_split["test"].shuffle(seed=123).map(self.tokenize_data)

#     #     return train_data, val_data


#     def prepare_data(self, data, val_set_size=100):
#         # Check if `data` is a valid Hugging Face dataset
#         print(f"Dataset before split: {data}")

#         # Filter out rows with missing 'text' or 'label' fields
#         data = data.filter(lambda example: example['text'] is not None and example['label'] is not None)

#         # Split dataset into training and validation sets
#         train_val_split = data.train_test_split(test_size=val_set_size, shuffle=True, seed=123)

#         # Debugging output to check dataset size
#         print(f"Training set size: {len(train_val_split['train'])}")
#         print(f"Validation set size: {len(train_val_split['test'])}")

#         # Shuffle and tokenize the training and validation sets
#         train_data = train_val_split["train"].shuffle(seed=123).map(self.tokenize_data, batched=True, batch_size=16)
#         val_data = train_val_split["test"].shuffle(seed=123).map(self.tokenize_data, batched=True, batch_size=16)

#         # Validate tokenized data
#         print(f"First tokenized example in train_data: {train_data[0]}")
#         print(f"First tokenized example in val_data: {val_data[0]}")

#         return train_data, val_data





#     def train_model(self, train_data, val_data, training_args):
#         self.model = self.model.to(self.device)
#         self.model.gradient_checkpointing_enable()
        
#         for name, module in self.model.named_modules():
#             print(name)        

#         lora_config = LoraConfig(
#             r=16,
#             lora_alpha=32,
#             target_modules=["q_proj", "k_proj", "v_proj"],
#             lora_dropout=0.05,
#             bias="none",
#             task_type="CAUSAL_LM",
#         )


#         self.model = get_peft_model(self.model, lora_config)
        
#         trainer = Trainer(
#             model=self.model,
#             train_dataset=train_data,
#             eval_dataset=val_data,
#             args=training_args,
#             # data_collator=DataCollatorForSeq2Seq(
#             #     self.tokenizer,
#             #     pad_to_multiple_of=8,
#             #     return_tensors="pt",
#             #     padding=True,
#             # ),
#             data_collator = DataCollatorForLanguageModeling(
#                 tokenizer=self.tokenizer,
#                 mlm=False,  # Set to False since you're doing causal language modeling
#                 pad_to_multiple_of=8
#                 )
#         )
        
#         self.model.config.use_cache = False
#         trainer.train()
#         self.model.save_pretrained(self.model_path)

#     def finetune(self, data_path, training_args):
#         print("LOADING DATASET")

#         data = Dataset.from_pandas(training_df[['concatenated_text', 'BIKE_CLE_TEXT']])
#         data = data.rename_column("concatenated_text", "text")
#         data = data.rename_column("BIKE_CLE_TEXT", "label")

#         print("DONE LOADING DATASET")
#         print("PREPARING DATASET")
#         train_data, val_data = self.prepare_data(data)
#         print("DONE PREPARING DATASET")
#         self.train_model(train_data, val_data, training_args)

# def lets_finetune(
#     device=DEVICE,
#     model=BASE_MODEL,
#     per_device_batch_size=1,
#     gradient_accumulation_steps=24,
#     warmup_steps=20,
#     learning_rate=2e-5,
#     max_steps=200,
# ):
#     print(f"\n{'='*60}")
#     print("Training Parameters:")
#     print(f"Foundation model: {BASE_MODEL}")
#     print(f"Model save path: {MODEL_PATH}")
#     print(f"Device used: {DEVICE}")
#     if DEVICE.type.startswith("xpu"):
#         print(f"Intel GPU: {torch.xpu.get_device_name()}")
#     print(f"Batch size per device: {per_device_batch_size}")
#     print(f"Gradient accum. steps: {gradient_accumulation_steps}")
#     print(f"Warmup steps: {warmup_steps}")
#     print(f"Max steps: {max_steps}")
#     print(f"Learning rate: {learning_rate}")
#     print(f"{'='*60}\n")

#     finetuner = FineTuner(base_model_id=model, model_path=MODEL_PATH, device=device)

#     training_args = TrainingArguments(
#         per_device_train_batch_size=per_device_batch_size,
#         gradient_accumulation_steps=gradient_accumulation_steps,
#         warmup_steps=warmup_steps,
#         max_steps=max_steps,
#         learning_rate=learning_rate,
#         bf16=True,
#         use_ipex=True,
#         logging_steps=20,
#         save_strategy="steps",
#         save_steps=20,
#         evaluation_strategy="steps",
#         eval_steps=20,
#         optim="adamw_hf",
#         output_dir=ADAPTER_PATH,
#         save_total_limit=3,
#         load_best_model_at_end=True,
#         ddp_find_unused_parameters=False,
#         group_by_length=True,
#         report_to="wandb" if ENABLE_WANDB else [],
#         fp16=False,
#     )

#     finetuner.finetune(DATA_PATH, training_args)




In [15]:
# if __name__ == "__main__":
#     lets_finetune()


Training Parameters:
Foundation model: mistralai/Mistral-7B-v0.1
Model save path: ./final_model
Device used: cpu
Batch size per device: 1
Gradient accum. steps: 24
Warmup steps: 20
Max steps: 200
Learning rate: 2e-05

Attempting to load model and tokenizer from: /home/common/data/Big_Data/GenAI/llm_models/mistralai--Mistral-7B-v0.1
Failed to load from /home/common/data/Big_Data/GenAI/llm_models/mistralai--Mistral-7B-v0.1. Attempting to download...


Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  8.74it/s]
/media/dzielinski06/HDD1/AI894 - Capstone/Complete Collision Recorder/.venv/lib/python3.12/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


LOADING DATASET
DONE LOADING DATASET
PREPARING DATASET
Dataset before split: Dataset({
    features: ['text', 'label', '__index_level_0__'],
    num_rows: 1315
})


Filter: 100%|██████████| 1315/1315 [00:00<00:00, 14206.74 examples/s]


Training set size: 1215
Validation set size: 100


In [9]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")

torch.cuda.empty_cache()

# Tokenize the input and labels
inputs = tokenizer(list(training_df['concatenated_text']), padding=True, max_length=512, return_tensors="pt")
labels = tokenizer(list(training_df['BIKE_CLE_TEXT']), padding=True, max_length=512, return_tensors="pt")

inputs = {key: value.to(device) for key, value in inputs.items()}
labels = {key: value.to(device) for key, value in labels.items()}

# Create a custom dataset
dataset = CustomDataset(inputs, labels)



NameError: name 'tokenizer' is not defined

In [ ]:
# Set batch size
batch_size = 1

# Define training arguments and enable FP16 for mixed precision
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=100,
    fp16=True  # Enable mixed precision
)

# Initialize the Hugging Face Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)

# Start training
trainer.train()


Training Parameters:
Foundation model: mistralai/Mistral-7B-v0.1
Model save path: ./final_model
Device used: cpu
Batch size per device: 1
Gradient accum. steps: 24
Warmup steps: 20
Max steps: 200
Learning rate: 2e-05

Attempting to load model and tokenizer from: /home/common/data/Big_Data/GenAI/llm_models/mistralai--Mistral-7B-v0.1
Failed to load from /home/common/data/Big_Data/GenAI/llm_models/mistralai--Mistral-7B-v0.1. Attempting to download...


Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  8.66it/s]
/media/dzielinski06/HDD1/AI894 - Capstone/Complete Collision Recorder/.venv/lib/python3.12/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


⋄ LOADING DATASET
⋄ DONE LOADING DATASET


https://huggingface.co/spikecodes/ai-911-operator/blob/main/dispatch.ipynb

In [11]:

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")
tokenizer.pad_token_id = 0
tokenizer.padding_side = "left"

# Enable gradient checkpointing to save memory
model.gradient_checkpointing_enable()
device = torch.device("cpu")
model.to(device)


# Tokenize the input and labels
# Set the eos_token as the padding token

inputs = tokenizer(list(training_df['concatenated_text']), 
                   padding=True, truncation=True, max_length=512, return_tensors="pt",
                   )
labels = tokenizer(list(training_df['BIKE_CLE_TEXT']), 
                   padding=True, truncation=True, max_length=512, return_tensors="pt",
                   )

# outputs = model(**inputs)
# print(f"Logits shape: {outputs.logits.shape}")  # (batch_size, sequence_length, vocab_size)
# print(f"Labels shape: {labels['input_ids'].shape}")  # (batch_size, sequence_length)

# Shift the labels for causal language models
# labels['input_ids'] = labels['input_ids'][:, 1:].contiguous()  # Shift labels by one token


# Create a custom dataset
dataset = CustomDataset(inputs, labels)

# Set batch size
batch_size = 1

# Define training arguments, explicitly disabling GPU usage (no_cuda=True)
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=1,  # Small batch size to fit in memory
    gradient_accumulation_steps=8,  # Accumulate gradients over 8 steps
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=100,
    label_smoothing_factor=0.1,  # Apply label smoothing
    no_cuda=True,  # Ensure training happens on CPU
    fp16=False     # Disable mixed precision since you're using CPU
)

# Initialize the Hugging Face Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)

# Start training
trainer.train()


/media/dzielinski06/HDD1/AI894 - Capstone/Complete Collision Recorder/.venv/lib/python3.12/site-packages/transformers/training_args.py:1560: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(
  0%|          | 0/492 [00:00<?, ?it/s]`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


RuntimeError: The size of tensor a (511) must match the size of tensor b (133) at non-singleton dimension 1

# GPU Deployment

In [38]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments

torch.cuda.empty_cache()


# Tokenize the input and labels
inputs = tokenizer(list(training_df['concatenated_text']), padding=True, max_length=512, return_tensors="pt")
labels = tokenizer(list(training_df['BIKE_CLE_TEXT']), padding=True, max_length=512, return_tensors="pt")

inputs = {key: value.to(device) for key, value in inputs.items()}
labels = {key: value.to(device) for key, value in labels.items()}

# Create a custom dataset
dataset = CustomDataset(inputs, labels)

# Set batch size
batch_size = 1

# Define training arguments and enable FP16 for mixed precision
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=100,
    fp16=True  # Enable mixed precision
)

# Initialize the Hugging Face Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)

# Start training
trainer.train()

/media/dzielinski06/HDD1/AI894 - Capstone/Complete Collision Recorder/.venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:2855: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


OutOfMemoryError: CUDA out of memory. Tried to allocate 224.00 MiB. GPU 0 has a total capacity of 5.78 GiB of which 111.88 MiB is free. Process 419769 has 2.96 GiB memory in use. Including non-PyTorch memory, this process has 1.76 GiB memory in use. Of the allocated memory 1.68 GiB is allocated by PyTorch, and 5.72 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [14]:
from transformers import AutoTokenizer

# Tokenize the input and labels


In [18]:
model.save_pretrained("./fine_tuned_PEFT_llm")
tokenizer.save_pretrained("./fine_tuned_PEFT_llm")


('./fine_tuned_gemini_llm/tokenizer_config.json',
 './fine_tuned_gemini_llm/special_tokens_map.json',
 './fine_tuned_gemini_llm/tokenizer.json')